<a href="https://colab.research.google.com/github/softmurata/colab_notebooks/blob/main/computervision/zeroshotobjectdection_groundingdino.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installation

In [ ]:
import os
HOME = "/content"
%cd {HOME}
!git clone https://github.com/IDEA-Research/GroundingDINO.git
%cd {HOME}/GroundingDINO
!pip install -q -e .
!pip install -q roboflow

CONFIG_PATH = os.path.join(HOME, "GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py")
print(CONFIG_PATH, "; exist:", os.path.isfile(CONFIG_PATH))

%cd {HOME}
!mkdir {HOME}/weights
%cd {HOME}/weights

!wget -q https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha/groundingdino_swint_ogc.pth
WEIGHTS_NAME = "groundingdino_swint_ogc.pth"
WEIGHTS_PATH = os.path.join(HOME, "weights", WEIGHTS_NAME)
print(WEIGHTS_PATH, "; exist:", os.path.isfile(WEIGHTS_PATH))

%cd {HOME}
!mkdir {HOME}/data
%cd {HOME}/data

!wget -q https://media.roboflow.com/notebooks/examples/dog.jpeg
!wget -q https://media.roboflow.com/notebooks/examples/dog-2.jpeg
!wget -q https://media.roboflow.com/notebooks/examples/dog-3.jpeg
!wget -q https://media.roboflow.com/notebooks/examples/dog-4.jpeg

%cd {HOME}/GroundingDINO

Import Libraries

In [4]:
import os
import supervision as sv
from PIL import Image
import numpy as np
import cv2

import pycocotools.mask as mask_util
from groundingdino.util.inference import load_model, load_image, predict, annotate

Load grounding dino model

In [5]:
HOME = "/content"
CONFIG_PATH = os.path.join(HOME, "GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py")
WEIGHTS_NAME = "groundingdino_swint_ogc.pth"
WEIGHTS_PATH = os.path.join(HOME, "weights", WEIGHTS_NAME)
detection_model = load_model(CONFIG_PATH, WEIGHTS_PATH)

/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


final text_encoder_type: bert-base-uncased


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Download test image

In [4]:
!wget -q https://media.roboflow.com/notebooks/examples/dog.jpeg -O /content/test.jpeg

Inference

In [ ]:
#@title dog
IMAGE_PATH = "/content/test.jpeg"

TEXT_PROMPT = "dog"  # rectangle, region, shape region, segment, fragment
BOX_TRESHOLD = 0.3  # 0.05~0.1 -> 部品点数とか図面の密度に応じている.
TEXT_TRESHOLD = 0.3  # 0.05~0.1

image_source, image = load_image(IMAGE_PATH)

boxes, logits, phrases = predict(
    model=detection_model, 
    image=image, 
    caption=TEXT_PROMPT, 
    box_threshold=BOX_TRESHOLD, 
    text_threshold=TEXT_TRESHOLD
)

annotated_frame = annotate(image_source=image_source, boxes=boxes, logits=logits, phrases=phrases)

%matplotlib inline  
sv.plot_image(annotated_frame, (16, 16))

In [ ]:
#@title building
IMAGE_PATH = "/content/test.jpeg"

TEXT_PROMPT = "building"  # rectangle, region, shape region, segment, fragment
BOX_TRESHOLD = 0.3  # 0.05~0.1 -> 部品点数とか図面の密度に応じている.
TEXT_TRESHOLD = 0.3  # 0.05~0.1

image_source, image = load_image(IMAGE_PATH)

boxes, logits, phrases = predict(
    model=detection_model, 
    image=image, 
    caption=TEXT_PROMPT, 
    box_threshold=BOX_TRESHOLD, 
    text_threshold=TEXT_TRESHOLD
)

annotated_frame = annotate(image_source=image_source, boxes=boxes, logits=logits, phrases=phrases)

%matplotlib inline  
sv.plot_image(annotated_frame, (16, 16))

Pipeline function

In [8]:
def inference_pipeline(image_path, text_prompt, box_threshold=0.3, text_threshold=0.3):
  image_source, image = load_image(image_path)

  boxes, logits, phrases = predict(
      model=detection_model, 
      image=image, 
      caption=text_prompt, 
      box_threshold=box_threshold, 
      text_threshold=text_threshold
  )

  annotated_frame = annotate(image_source=image_source, boxes=boxes, logits=logits, phrases=phrases)

  %matplotlib inline  
  sv.plot_image(annotated_frame, (16, 16))


Advanced Inference

In [ ]:
!wget https://prtimes.jp/i/1355/5420/resize/d1355-5420-149832-0.jpg -O /content/soccer.jpg

In [ ]:
inference_pipeline("/content/soccer.jpg", "ball", 0.2, 0.2)

In [ ]:
inference_pipeline("/content/soccer.jpg", "player,ball", 0.2, 0.2)

In [ ]:
# soccer 2
!wget https://prtimes.jp/i/31288/7/resize/d31288-7-332931-0.png -O /content/soccer.png

In [ ]:
inference_pipeline("/content/soccer.png", "goal", 0.2, 0.2)

In [ ]:
#@title intersection
!wget https://camera-map.com/wp-content/uploads/ann-shibuya.jpg -O /content/intersection.jpg

In [ ]:
inference_pipeline("/content/intersection.jpg", "car", 0.17, 0.17)